# To do list

+ Mapas por equipos -> Vicente
+ stats ftu por equipos -> Aldo
+ Guardar los DataFrames en un CSV -> Aldo

In [1]:
# Importamos la clase HltvScraper del archivo scraper.py
from clase_scraper import HltvScraper

# Creamos una instancia de la clase
hltv_scraper = HltvScraper()

## Método ``players_of_team``

El método se encarga de obtener todo los obtener las ids de los jugadores que componen al equipo conultado

In [2]:
dicc = hltv_scraper.players_of_team("/4608/natus-vincere")
dicc

{'Aleksib': '/stats/players/9816/aleksib?startDate=2023-11-15&endDate=2024-11-15&matchType=all&maps=all&rankingFilter=all',
 'iM': '/stats/players/14759/im?startDate=2023-11-15&endDate=2024-11-15&matchType=all&maps=all&rankingFilter=all',
 'b1t': '/stats/players/18987/b1t?startDate=2023-11-15&endDate=2024-11-15&matchType=all&maps=all&rankingFilter=all',
 'jL': '/stats/players/19206/jl?startDate=2023-11-15&endDate=2024-11-15&matchType=all&maps=all&rankingFilter=all',
 'w0nderful': '/stats/players/20127/w0nderful?startDate=2023-11-15&endDate=2024-11-15&matchType=all&maps=all&rankingFilter=all'}

## Método ``individual_stats``

El método scrapea las stats individuales del jugador consulatado

In [3]:
df1 = hltv_scraper.individual_stats("/9816/aleksib")
df1

Overall stats                                                     \
                Kills Deaths Kill / Death Kill / Round Rounds with kills   
aleksib          1985   2407         0.82         0.53              1503   

                                      Opening stats                       \
        Kill - Death difference Total opening kills Total opening deaths   
aleksib                    -422                 287                  377   

                                                ...   Round stats  \
        Opening kill ratio Opening kill rating  ... 2 kill rounds   
aleksib               0.76                0.88  ...           326   

                                                  Weapon stats               \
        3 kill rounds 4 kill rounds 5 kill rounds  Rifle kills Sniper kills   
aleksib            67             6             1         1324           19   

                                              
        SMG kills Pistol kills Grenade Other  
aleksib       259          320      64    20  

[1 rows x 24 columns]

## Método ``stats_players_team``

El método llama a ``players_of_team`` para obtener a los jugadores del equipo consultado, para así llamar a ``individual_stats`` en cada uno y luego retornar un DataFrame con todas las stats individuales del equipo.

In [4]:
df_jugadores_navi = hltv_scraper.stats_players_team("/4608/natus-vincere")
df_jugadores_navi

Overall stats                                                     \
                  Kills Deaths Kill / Death Kill / Round Rounds with kills   
aleksib            1985   2407         0.82         0.53              1503   
im                 2611   2501         1.04         0.70              1761   
b1t                2718   2426         1.12         0.73              1846   
jl                 2670   2425         1.10         0.71              1792   
w0nderful          2701   2172         1.24         0.72              1849   

                                        Opening stats                       \
          Kill - Death difference Total opening kills Total opening deaths   
aleksib                      -422                 287                  377   
im                            110                 460                  434   
b1t                           292                 436                  359   
jl                            245                 412                  358   
w0nderful                     529                 386                  229   

                                                  ...   Round stats  \
          Opening kill ratio Opening kill rating  ... 2 kill rounds   
aleksib                 0.76                0.88  ...           326   
im                      1.06                1.09  ...           491   
b1t                     1.21                1.09  ...           509   
jl                      1.15                1.06  ...           484   
w0nderful               1.69                1.08  ...           413   

                                                    Weapon stats               \
          3 kill rounds 4 kill rounds 5 kill rounds  Rifle kills Sniper kills   
aleksib              67             6             1         1324           19   
im                  140            25             1         2087           13   
b1t                 142            25             1         2037           22   
jl                  154            26             2         2037           19   
w0nderful           168            29             4         1080         1100   

                                                
          SMG kills Pistol kills Grenade Other  
aleksib         259          320      64    20  
im               72          420      20    14  
b1t             198          440      23    12  
jl              142          441      27    25  
w0nderful        40          465      21    15  

[5 rows x 24 columns]

## Método ``teams_major_qualifier``

Retorna una lista con los equipos participantes del major separados por grupos del qualifier.

In [5]:
lista = hltv_scraper.teams_major_qualifier()

In [6]:
europa_1 = lista[0]
europa_2 = lista[1]
america_rmr = lista[2]
asia_rmr = lista[3]

asia_rmr

{'The MongolZ': '/6248/The%20MongolZ',
 'FlyQuest': '/12774/FlyQuest',
 'Lynn Vision': '/8840/Lynn%20Vision',
 'Rare Atom': '/11514/Rare%20Atom',
 'TALON': '/8210/TALON',
 'GR': '/12088/GR',
 'Alter Ego': '/9958/Alter%20Ego',
 'DRILLAS': '/12937/DRILLAS'}

## Método ``career_player``

Retorna un DataFrame con las estadísticas anuales del Career rating 1.0 del jugador consultado

In [7]:
df = hltv_scraper.career_player("/9816/aleksib")
df

,Period,All,Online,LAN,Majors
0,2015,1.08 (5),NaN,1.08 (5),NaN
0,2016,1.00 (51),0.96 (28),1.04 (23),NaN
0,2017,1.09 (184),1.05 (135),1.19 (49),NaN
0,2018,0.98 (224),0.93 (128),1.05 (96),NaN
0,2019,0.94 (137),0.92 (17),0.94 (120),1.00 (29)
0,2020,0.90 (220),0.90 (203),0.88 (17),NaN
0,2021,0.92 (133),0.91 (112),0.94 (21),NaN
0,2022,0.87 (102),0.95 (8),0.86 (94),0.85 (18)
0,2023,0.88 (163),0.86 (37),0.89 (126),1.01 (12)
0,2024,0.82 (155),0.78 (4),0.82 (151),0.74 (18)


## Método ``pistol_rounds``

Retorna un DataFrame con las estadísticas anuales de los equipos para las rondas de pistolas.

In [8]:
df = hltv_scraper.pistol_rounds("both")
df

,Team,Maps,Won - Lost,Pistol win %,Round 2 conv,Round 2 break
0,NRG,221,240 - 202,54.3%,88.5%,15.8%
1,M80,231,240 - 222,51.9%,84.9%,24.9%
2,Spirit,162,174 - 150,53.7%,90.7%,10.0%
3,The MongolZ,211,250 - 172,59.2%,84.2%,19.2%
4,Imperial,286,309 - 263,54.0%,86.1%,13.0%
...,...,...,...,...,...,...
80,Verdant,155,149 - 161,48.1%,77.7%,14.9%
81,Illuminar,219,200 - 238,45.7%,85.5%,14.3%
82,Rhyno,161,145 - 177,45.0%,89.0%,15.3%
83,EYEBALLERS,307,296 - 318,48.2%,82.8%,14.2%


## Método ``team_stats_by_map``

Retorna un DataFrame con las estadísticas del equipo en un mapa específico.

In [2]:
df_dust2 = hltv_scraper.team_stats_by_map("/4608/natus-vincere", "Dust2")
df_dust2

,Times played,Wins / draws / losses,Total rounds played,Rounds won,Win percent,Pistol rounds,Pistol rounds won,Pistol round win percent,CT round win percent,T round win percent
0,16,13 / 0 / 3,341,196,81.2%,32,19,59.4%,58.5%,56.6%


## Método ``all_teams_stats_by_map``

In [2]:
df_dust2 = hltv_scraper.all_teams_stats_by_map("Dust2")
df_dust2

,Times played,Wins / draws / losses,Total rounds played,Rounds won,Win percent,Pistol rounds,Pistol rounds won,Pistol round win percent,CT round win percent,T round win percent,group
0,16,13 / 0 / 3,341,196,81.2%,32,19,59.4%,58.5%,56.6%,europa_1
0,22,15 / 0 / 7,443,259,68.2%,44,26,59.1%,54.7%,61.6%,europa_1
0,13,7 / 0 / 6,267,134,53.8%,26,12,46.2%,47.2%,54.6%,europa_1
0,16,8 / 0 / 8,338,172,50.0%,32,15,46.9%,54.1%,48.1%,europa_1
0,19,10 / 0 / 9,442,224,52.6%,38,16,42.1%,53.6%,47.7%,europa_1
0,12,8 / 0 / 4,229,129,66.7%,24,14,58.3%,56.2%,56.5%,europa_1
0,13,7 / 0 / 6,291,150,53.8%,26,15,57.7%,47.5%,55.3%,europa_1
0,11,5 / 0 / 6,235,112,45.5%,22,6,27.3%,60.0%,35.8%,europa_1
0,11,2 / 0 / 9,223,90,18.2%,22,8,36.4%,44.4%,34.4%,europa_1
0,37,20 / 0 / 17,782,401,54.1%,74,43,58.1%,48.4%,53.9%,europa_1
